In [1]:
import numpy as np

from skimage import io
from skimage import filters
from skimage.transform import resize
from skimage.morphology import binary_opening, binary_closing, ball
from scipy.ndimage.morphology import binary_fill_holes

import os
import napari
from napari.types import ImageData

from magicgui import magicgui
import morphotrack.binary
# https://napari.org/magicgui/examples/napari/napari_parameter_sweep.html

/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
nuclei = io.imread('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/R02_ch488_10um.tif')

masks = {
    'white_matter': io.imread('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/white_matter_labkit.tif')==1,
    'layer1': io.imread('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/l1_labkit.tif')==1,
    'tissue': io.imread('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/l1_labkit.tif')!=2
}

In [3]:
mask = masks['white_matter'].copy()
mask = morphotrack.binary.extract_largest_object_from_binary(mask)
mask = binary_fill_holes(mask)

In [5]:
@magicgui(auto_call=True,sigma={"widget_type": "FloatSlider", "max": 10},mode={"choices": ["reflect", "constant", "nearest", "mirror", "wrap"]},layout='horizontal')
def gaussian_blur(layer: ImageData, sigma: float = 2.0, mode="nearest") -> ImageData:
    """Apply a gaussian blur to ``layer``."""
    if layer is not None:
        return filters.gaussian(layer, sigma=sigma, mode=mode)>=0.5

In [6]:
viewer = napari.Viewer()
viewer.add_image(mask, name="image")

viewer.window.add_dock_widget(gaussian_blur)
viewer.layers.events.changed.connect(gaussian_blur.reset_choices)

napari.run()

In [6]:
mask_gauss = filters.gaussian(mask, sigma=3, mode='nearest')>=0.5
# mask_gauss = morphotrack.binary.extract_largest_object_from_binary(mask_gauss)

In [8]:
mask_gauss_opened = binary_opening(mask_gauss,ball(5))
mask_gauss_opened = morphotrack.binary.extract_largest_object_from_binary(mask_gauss_opened)

In [7]:
viewer = napari.Viewer()
viewer.add_image(nuclei, name="orig",blending='additive',colormap='gray')
viewer.add_image(mask, name="image1",blending='additive',colormap='magenta')
viewer.add_image(mask_gauss, name="image2",blending='additive',colormap='green')

<Image layer 'image2' at 0x7f7b6978e340>

In [8]:
io.imsave('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/white_matter.tif',mask_gauss)

/tmp/ipykernel_2820542/2784746542.py:1: UserWarning: /mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/white_matter.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/morphotrack/white_matter.tif',mask_gauss)


In [12]:
#